# Load Dataset with inital features

In [1]:
import pandas as pd
import azureml.core
from azureml.core import Dataset, Datastore, Workspace

ws = Workspace.from_config()

qualitydf = Dataset.get_by_name(workspace=ws,name="iiot_quality_featured_data").to_pandas_dataframe()
print("Rows => {0}".format(qualitydf.shape[0]))
print("Columns => {0}".format(qualitydf.shape[1]))
qualitydf.head()

Rows => 7711
Columns => 20


,Quality,S16,S20,S19,S18,S29,S41,S9,S10,S8,S11,S14,S13,S28,S15,S26,S33,S7,S3,S39
0,1,22.02698,33.14618,35.47309,31.48723,7.497666,7221.264648,36.1,39.5,27.6,21.9,2089.468,2046.699,26.79577,943.1278,31.57756,40.0,23.57390,29.04669,5773.366
1,1,22.02698,33.14618,35.47309,31.31926,7.491282,7213.028320,35.9,39.4,27.7,21.8,2069.875,2052.430,26.97901,930.0323,32.33240,40.0,23.60346,28.67866,5773.366
2,1,22.36291,32.97821,35.47309,31.65519,7.484899,7229.500977,35.9,39.5,28.1,21.9,2075.162,2060.232,26.99349,930.1899,32.49136,40.0,23.57944,28.74189,5773.366
3,1,22.19494,32.97821,35.47309,31.48723,7.500858,7213.028320,35.9,39.6,28.4,21.9,2137.928,2046.037,27.02537,943.1278,31.77274,40.0,23.60447,27.57296,5773.366
4,1,22.02698,32.97821,35.47309,31.65519,7.497666,7229.500977,36.1,39.6,28.8,22.1,2090.136,2033.450,27.14663,943.4471,31.68100,40.0,23.64226,28.34144,5789.014


# Split the data for model evaluation

In [2]:
X = qualitydf.drop(['Quality'], axis=1)   # select all sensor values
y = qualitydf["Quality"]  # select the target column to predict

# Split data into train(70%) and test(30%) datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,random_state=0,stratify=y)

# Build baseline model 

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

# Replace null values with "median" and normalize all numeric values using MinMaxScaler
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', MinMaxScaler())])
transformations = ColumnTransformer(transformers=[('num', numeric_transformer, X.columns)])

# Build classifier pipeline with preprocessing steps as above, and logistic regression as the model
# Only use training dataset for build the model
classifierPipeline = Pipeline(steps=[('preprocessor', transformations),('classifier', LogisticRegression())])
model = classifierPipeline.fit(X_train, y_train)

# Run the model and extract predictions
y_pred = classifierPipeline.predict(X_test)
y_pred_train = classifierPipeline.predict(X_train)

# Score the model against true values
print('Training set score: {:.4f}'.format(classifierPipeline.score(X_train, y_train)))
print('Test set score: {:.4f}'.format(classifierPipeline.score(X_test, y_test)))

print(classification_report(y_test, y_pred))

Training set score: 0.8205
Test set score: 0.8202
              precision    recall  f1-score   support

           0       1.00      0.00      0.00       417
           1       0.82      1.00      0.90      1897

    accuracy                           0.82      2314
   macro avg       0.91      0.50      0.45      2314
weighted avg       0.85      0.82      0.74      2314



# Try multiple models and compare all the scores

In general we are looking for a model with high "precision" and high "recall". Depending on the risk appetite of the business and the consequences of the actions that we can recommend based on the predictions, we may choose to focus on specific evaluation metrics and thresholds. 

See [Classification metrics](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-understand-automated-ml?msclkid=a25fa0facff911ecae2ae07854ffb705#classification-metrics) for more details

In [62]:
def add_classification_report(classifername, reportdf, classvalues):
    report_data = []
    for c in classvalues:
        row = {}
        row["classifer"] = classifername
        row['class'] = c
        row['precision'] = float(reportdf["precision"][c])
        row['recall'] = float(reportdf["recall"][c])
        row['f1-score'] = float(reportdf["f1-score"][c])
        row['accuracy'] = float(reportdf["precision"]["accuracy"])
        row['support'] = float(reportdf["support"][c])
        report_data.append(row)
    return pd.DataFrame.from_dict(report_data)

In [63]:
# Try multiple classifiers
classifierList = [DecisionTreeClassifier(), RandomForestClassifier(),LGBMClassifier(), 
                  AdaBoostClassifier(), GradientBoostingClassifier(), 
                  LogisticRegression(), RidgeClassifier(), SGDClassifier()]

allreportsdf = pd.DataFrame()

for currentClassifier in classifierList:
    currentPipeline = Pipeline(steps=[('preprocessor', transformations),('classifier', currentClassifier)])
    model = currentPipeline.fit(X_train, y_train)
    y_pred = currentPipeline.predict(X_test)
    y_pred_train = currentPipeline.predict(X_train)
    currentreport = classification_report(y_test, y_pred,output_dict=True)
    currentreportdf = pd.DataFrame(currentreport).transpose()
    reportdf = add_classification_report(type(currentClassifier).__name__, currentreportdf,[0,1])
    if (len(allreportsdf) == 0):
        allreportsdf = reportdf
    else:
        allreportsdf = allreportsdf.append(reportdf, ignore_index = True)

allreportsdf.head(50)

,classifer,class,precision,recall,f1-score,accuracy,support
0,DecisionTreeClassifier,0,0.314815,0.326139,0.320377,0.750648,417.0
1,DecisionTreeClassifier,1,0.850691,0.843964,0.847314,0.750648,1897.0
2,RandomForestClassifier,0,0.797468,0.151079,0.254032,0.840104,417.0
3,RandomForestClassifier,1,0.841611,0.991566,0.910455,0.840104,1897.0
4,LGBMClassifier,0,0.773810,0.155875,0.259481,0.839672,417.0
5,LGBMClassifier,1,0.842152,0.989984,0.910104,0.839672,1897.0
6,AdaBoostClassifier,0,0.555556,0.023981,0.045977,0.820657,417.0
7,AdaBoostClassifier,1,0.822735,0.995783,0.901026,0.820657,1897.0
8,GradientBoostingClassifier,0,0.775862,0.107914,0.189474,0.833621,417.0
9,GradientBoostingClassifier,1,0.835106,0.993147,0.907296,0.833621,1897.0
